In [ ]:
import os
import requests
from tqdm.notebook import tqdm
import zipfile
import xarray as xr
import numpy as np
import rioxarray
import xtrude
import pydeck as pdk
import matplotlib.pyplot as plt
from ipywidgets import Output

In [ ]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = os.path.join(name, name, 'w001001.adf')

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024)):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

In [ ]:
da = rioxarray.open_rasterio(adffile, masked=True)
da = da.sel(band=1)
da = xr.where(np.isnan(da), 0, da)
da.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
debug_output = Output()
debug_output

In [ ]:
# texture is a color map of the elevation:
l = da.xtrude.plot(colormap=plt.cm.terrain, terrain_port=8050, surface_port=8060, debug_output=debug_output)

# texture is an on-line basemap:
#l = da.xtrude.plot(surface_url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', debug_output=debug_output)

In [ ]:
view_state = pdk.ViewState(latitude=-14, longitude=-72, zoom=5, bearing=0, pitch=0)
r = pdk.Deck(l, initial_view_state=view_state)
r.show()